In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
SPREADSHEET_ID = os.getenv("EXPENSES_SPREADHSEET_ID")

In [ ]:
months = ["enero", "febrero", "marzo", "abril", "mayo", "junio", "julio", "agosto", "septiembre", "octubre", "noviembre", "diciembre"]
month_urls = {month: f"https://docs.google.com/spreadsheets/d/{SPREADSHEET_ID}/gviz/tq?tqx=out:csv&sheet={month}" for month in months}

grupos = {
    "fijos": ["alquileres", "servicios_esenciales", "servicios_no_esenciales"],
    "corrientes": ["hogar", "transporte", "salidas"],
    "irregulares": ["shopping", "otros"],
}

In [ ]:
totales_mensuales = []

for mes in months:
    url = month_urls[mes]
    df = pd.read_csv(
        url, usecols=grupos["fijos"] + grupos["corrientes"] + grupos["irregulares"]
    )
    fila_total = df.iloc[-1]
    fila_limpia = {
        cat: pd.to_numeric(
            fila_total[cat].replace("$", "").replace(",", "").strip(), errors="coerce"
        )
        for cat in grupos["fijos"] + grupos["corrientes"] + grupos["irregulares"]
    }
    fila_limpia["mes"] = mes
    totales_mensuales.append(fila_limpia)
df_totales = pd.DataFrame(totales_mensuales).set_index("mes")
df_totales.tail(12)

In [ ]:
def plot_grupo(grupo, nombre):
    df_grupo = df_totales[grupo]
    df_grupo.plot(marker="o", figsize=(10, 5))
    plt.title(f"{nombre} por Mes")
    plt.ylabel("Monto en ARS")
    plt.xlabel("Mes")
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.show()


plot_grupo(grupos["fijos"], "Gastos Fijos")
plot_grupo(grupos["corrientes"], "Gastos Corrientes")
plot_grupo(grupos["irregulares"], "Gastos Irregulares")

trimestres = {month: f"Q{(i // 3) + 1}" for i, month in enumerate(months)}

df_totales["trimestre"] = df_totales.index.map(trimestres)

promedios_trimestrales = {}
for nombre_grupo, categorias in grupos.items():
    df_prom = df_totales.groupby("trimestre")[categorias].mean()
    promedios_trimestrales[nombre_grupo] = df_prom

for nombre_grupo, df_prom in promedios_trimestrales.items():
    df_prom.plot(kind="bar", stacked=True, figsize=(8, 5))
    plt.title(f"Promedio Trimestral - Grupo {nombre_grupo.capitalize()}")
    plt.ylabel("Monto promedio por categoría (ARS)")
    plt.xlabel("Trimestre")
    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [ ]:
datos_personales = pd.read_csv("../data/sueldos.csv")
datos_personales_del_anio = datos_personales.iloc[-len(months):]
total_anual = df_totales.sum(numeric_only=True).sum()
ingreso_anual = datos_personales_del_anio["sueldo_neto_ars"].sum()
ahorro_uso_discrecional = ingreso_anual - total_anual
porcentaje_gastado = total_anual / ingreso_anual * 100

print(f"Gasto total del año: ${total_anual:,.0f} ARS")
print(f"Ingreso neto del año: ${ingreso_anual:,.0f} ARS")
print(f"Diferencia: ${ahorro_uso_discrecional:,.0f} ARS")
print(f"Porcentaje del ingreso gastado: {porcentaje_gastado:.1f}%")

In [ ]:
gasto_por_categoria = df_totales.sum(numeric_only=True)

gasto_por_categoria_con_ahorro = gasto_por_categoria.copy()
gasto_por_categoria_con_ahorro["ahorro"] = ahorro_uso_discrecional

labels = gasto_por_categoria_con_ahorro.index.tolist()
values = gasto_por_categoria_con_ahorro.values
percentages = values / ingreso_anual * 100

In [ ]:
porcentajes_ideales = {
    "alquileres": 30,  # checked
    "servicios_esenciales": 7,  # checked
    "servicios_no_esenciales": 5,  # checked
    "hogar": 15,  # checked
    "transporte": 5,  # checked
    "salidas": 8,  # checked
    "shopping": 7,  # checked
    "otros": 3,  # checked
    "ahorro": 20,  # checked
}
print(sum(porcentajes_ideales.values()))
assert abs(sum(porcentajes_ideales.values()) - 100) < 1e-6, "El total no suma 100%"

labels_ideales = list(porcentajes_ideales.keys())
values_ideales = list(porcentajes_ideales.values())

In [ ]:
mes_actual = df_totales.index[-1]
gastos_mes_actual = df_totales.loc[mes_actual]
ingreso_mes_actual = datos_personales_del_anio["sueldo_neto_ars"].iloc[-1]
ahorro_mes_actual = ingreso_mes_actual - gastos_mes_actual.drop("trimestre").sum()
gastos_mes_actual_con_ahorro = gastos_mes_actual.drop("trimestre").copy()
gastos_mes_actual_con_ahorro["ahorro"] = ahorro_mes_actual

labels_mes_actual = gastos_mes_actual_con_ahorro.index.tolist()
values_mes_actual = gastos_mes_actual_con_ahorro.values
percentages_mes_actual = values_mes_actual / ingreso_mes_actual * 100

In [ ]:
fig, axes = plt.subplots(nrows=3, figsize=(10, 20))

wedges1, texts1, autotexts1 = axes[0].pie(
    percentages,
    autopct="%1.1f%%",
    startangle=90,
    wedgeprops={"edgecolor": "white"},
    textprops={"fontsize": 10},
)

axes[0].legend(
    wedges1,
    labels,
    title="Categorías",
    loc="center left",
    bbox_to_anchor=(1, 0.5),
    fontsize=10,
)
axes[0].set_title(f"Distribución Real del Ingreso Anual de ${ingreso_anual:,.0f} ARS")

wedges2, texts2, autotexts2 = axes[1].pie(
    values_ideales,
    autopct="%1.1f%%",
    startangle=90,
    wedgeprops={"edgecolor": "white"},
    textprops={"fontsize": 10},
)

axes[1].legend(
    wedges2,
    labels_ideales,
    title="Categorías",
    loc="center left",
    bbox_to_anchor=(1, 0.5),
    fontsize=10,
)
axes[1].set_title("Distribución Ideal del Ingreso Anual")

wedges3, texts3, autotexts3 = axes[2].pie(
    percentages_mes_actual,
    autopct="%1.1f%%",
    startangle=90,
    wedgeprops={"edgecolor": "white"},
    textprops={"fontsize": 10},
)
axes[2].legend(
    wedges3,
    labels_mes_actual,
    title="Categorías",
    loc="center left",
    bbox_to_anchor=(1, 0.5),
    fontsize=10,
)
axes[2].set_title(f"Distribución del Ingreso de {mes_actual.capitalize()}: ${ingreso_mes_actual:,.0f} ARS")

plt.tight_layout()
plt.show()
